In [64]:
# Compute free energy change of 8_pair long ssDNA activator (160 nt)
# It should be noted that in a computed structure of a targeting region, the left hybridized base may not always correspond to a right hybridized base as it may hybridize with a base outside of the targeting region
# In a computed structure, '(' refers to a left paired base and ")" refers to a right paired base

from nupack import *   # Import NUPACK package 

# Count the number of left base pairs and right base pairs in a long ssDNA structure
def paren_num(structure):
    left_paren_num = 0
    right_paren_num = 0
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_num += 1
        if structure[i] == ')':
            right_paren_num += 1
            
    return left_paren_num, right_paren_num

# Record the position of left paired bases and right paired bases in a long ssDNA structure
def paren_position(structure):
    left_paren = []
    right_paren = []
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            
    return left_paren, right_paren
        
# Record positions of base pairs with the order of base positions in "left_paren" corresponding to the base positions in "right_paren"
def find_pair(structure):
    left_paren = []
    right_paren = []
    left_paren_pre = []

    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_pre.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            left_paren.append(left_paren_pre[-1])
            left_paren_pre = left_paren_pre[:-1]

    return left_paren, right_paren

# 
def DNA_unwind_state(structure, i):
    left_paren, right_paren = find_pair(structure)
    ori_structure_part = structure[i*20:(i+1)*20]    # Compute the original structure of targeting region
    left_paren_num_part = paren_num(ori_structure_part)[0]   # Get the number of left base pairs
    right_paren_num_part = paren_num(ori_structure_part)[1]  # Get the number of right base pairs
    left_paren_part = paren_position(ori_structure_part)[0]  # Get the relative positions of left paired bases in 20-nt target
    right_paren_part = paren_position(ori_structure_part)[1] # Get the relative positions of right paired bases in 20-nt target

    left_paren_part = [x+i*20 for x in left_paren_part]      # Get the absolute positions of left paired bases 
    right_paren_part = [x+i*20 for x in right_paren_part]    # Get the absolute positions of right paired bases 
    left_paren_part = set(left_paren_part)                   # Arrange the order and eliminate repeats 
    right_paren_part = set(right_paren_part)
    index_left = [m for m, n in enumerate(left_paren) if n in left_paren_part]     # Get the order of left paired bases in the complete long ssDNA sequence
    index_right = [m for m, n in enumerate(right_paren) if n in right_paren_part]  # Get the order of right paired bases in the complete long ssDNA sequence
    for k1 in index_left:
        right_position = right_paren[k1]
        structure = structure[0:right_position] + '.' + structure[right_position+1:]    # Correct the left paired base to an unpaired base in the complete sequence
    for k2 in index_right:
        left_position = left_paren[k2]
        structure = structure[0:left_position] + '.' + structure[left_position+1:]      # Correct the right paired base to an unpaired base in the complete sequence
    
    # Correct the structure of targeting region to an unwinded state ready for hybridization to crRNA
    if i == 0:
        structure = '....................' + structure[20:] 
    else:
        structure = structure[0: i*20] + '....................' + structure[(i+1)*20:]
        
    return structure

# We adopted a weighted model to compute the free energy change of the targeting region of a long strand
# To be specific, the free energy change of targeting region in each suboptimal structure is individually considered and the ensemble energy change is the weighted sum of all energy changes in suboptimal structures
def main(sequence, unwind_domain):
    my_model = Model(material='DNA', ensemble='stacking', celsius=37, sodium=0.1, magnesium=0.01)   # Define NUPACK model, ion concentration in consistent with NEBuffer r3.1
    partition_function = pfunc(strands=sequence, model=my_model)
    mfe_structures = mfe(strands=sequence, model=my_model)    # Compute MFE structure of long strand

    subopt_structures = subopt(strands=sequence, energy_gap=2.91, model=my_model)   # Compute all suboptimal structures whose energies are close to the MFE (not exceeding 2.91 kcal/mol and in this case 99% of the suboptimal structures will be considered)
    prob_sum = 0   # Initialize the sum of frequency of a thermodynamic ensemble
    sum_weighted_DeltaG_unwind = 0   # Initialize the weighted sum of free energy change of targeting region 
    for i in range (0, len(subopt_structures)):
        # Compute the suboptimal structure 
        subopt_struct_candidate = str(subopt_structures[i].structure)   
        # Compute the energy under this suboptimal structure 
        subopt_energy_candidate = subopt_structures[i].energy  
        # Compute the frequency of this suboptimal structure 
        probability = structure_probability(strands=sequence, structure=subopt_struct_candidate, model=my_model)
        
        prob_sum += probability  # Frequency sum up
        # Compute suboptimal structure with the targeting region unwinded 
        subopt_struct_unwind = str(DNA_unwind_state(subopt_struct_candidate, unwind_domain))
        # Compute the energy of this suboptimal structure with the targeting region unwinded 
        subopt_energy_unwind = structure_energy(strands=sequence, structure=subopt_struct_unwind, model=my_model)
        
        # The free energy change is the difference of energy between the original state and the unwinded state
        deltaG_unwind = subopt_energy_candidate - subopt_energy_unwind
        # This energy change should be weighted with consideration of the frequency of this suboptimal structure 
        weighted_deltaG_unwind = deltaG_unwind * probability
        # Weighted free energy change sum up
        sum_weighted_DeltaG_unwind += weighted_deltaG_unwind
    
    print('This is domain: ' + str(unwind_domain))    # Refer to the index of target in a long strand
    print("Targeting region sequence: " + str(sequence[unwind_domain*20:(unwind_domain+1)*20]))
    print('Probability sum of all suboptimal strctures searched: ' + str(prob_sum)) 
    print('Weighted sum of energy needed to unwind this domain: ' + str(sum_weighted_DeltaG_unwind) + " kcal/mol")

# All the sequences of long ssDNA used
long_DNA_8_pair = 'AGTATATCTATTGATATACTCCATAAACTACAGTTCATGGGGGATCACGTAAGCGATCCCCCGTGGACGGCAGCCCACGGATAATGATGATAAAGTTTGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG'

# Return the results
print('Long ssDNA target sequence: ' + str(long_DNA_8_pair))
for i in range (0, 8):
    main(long_DNA_8_pair, i)
       

Long ssDNA target sequence: AGTATATCTATTGATATACTCCATAAACTACAGTTCATGGGGGATCACGTAAGCGATCCCCCGTGGACGGCAGCCCACGGATAATGATGATAAAGTTTGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG
This is domain: 0
Targeting region sequence: AGTATATCTATTGATATACT
Probability sum of all suboptimal strctures searched: 0.9921054497695768
Weighted sum of energy needed to unwind this domain: -5.440382170353726 kcal/mol
This is domain: 1
Targeting region sequence: CCATAAACTACAGTTCATGG
Probability sum of all suboptimal strctures searched: 0.9921054497695768
Weighted sum of energy needed to unwind this domain: -3.3596172028129723 kcal/mol
This is domain: 2
Targeting region sequence: GGGATCACGTAAGCGATCCC
Probability sum of all suboptimal strctures searched: 0.9921054497695768
Weighted sum of energy needed to unwind this domain: -5.150360412818166 kcal/mol
This is domain: 3
Targeting region sequence: CCGTGGACGGCAGCCCACGG
Probability sum of all suboptimal strctures searched: 0.9921054497695768
Weighted su

In [63]:
# Compute free energy change of 7_pair long ssDNA activator (160 nt)
# It should be noted that in a computed structure of a targeting region, the left hybridized base may not always correspond to a right hybridized base as it may hybridize with a base outside of the targeting region
# In a computed structure, '(' refers to a left paired base and ")" refers to a right paired base

from nupack import *   # Import NUPACK package 

# Count the number of left base pairs and right base pairs in a long ssDNA structure
def paren_num(structure):
    left_paren_num = 0
    right_paren_num = 0
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_num += 1
        if structure[i] == ')':
            right_paren_num += 1
            
    return left_paren_num, right_paren_num

# Record the position of left paired bases and right paired bases in a long ssDNA structure
def paren_position(structure):
    left_paren = []
    right_paren = []
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            
    return left_paren, right_paren
        
# Record positions of base pairs with the order of base positions in "left_paren" corresponding to the base positions in "right_paren"
def find_pair(structure):
    left_paren = []
    right_paren = []
    left_paren_pre = []

    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_pre.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            left_paren.append(left_paren_pre[-1])
            left_paren_pre = left_paren_pre[:-1]

    return left_paren, right_paren

# 
def DNA_unwind_state(structure, i):
    left_paren, right_paren = find_pair(structure)
    ori_structure_part = structure[i*20:(i+1)*20]    # Compute the original structure of targeting region
    left_paren_num_part = paren_num(ori_structure_part)[0]   # Get the number of left base pairs
    right_paren_num_part = paren_num(ori_structure_part)[1]  # Get the number of right base pairs
    left_paren_part = paren_position(ori_structure_part)[0]  # Get the relative positions of left paired bases in 20-nt target
    right_paren_part = paren_position(ori_structure_part)[1] # Get the relative positions of right paired bases in 20-nt target

    left_paren_part = [x+i*20 for x in left_paren_part]      # Get the absolute positions of left paired bases 
    right_paren_part = [x+i*20 for x in right_paren_part]    # Get the absolute positions of right paired bases 
    left_paren_part = set(left_paren_part)                   # Arrange the order and eliminate repeats 
    right_paren_part = set(right_paren_part)
    index_left = [m for m, n in enumerate(left_paren) if n in left_paren_part]     # Get the order of left paired bases in the complete long ssDNA sequence
    index_right = [m for m, n in enumerate(right_paren) if n in right_paren_part]  # Get the order of right paired bases in the complete long ssDNA sequence
    for k1 in index_left:
        right_position = right_paren[k1]
        structure = structure[0:right_position] + '.' + structure[right_position+1:]    # Correct the left paired base to an unpaired base in the complete sequence
    for k2 in index_right:
        left_position = left_paren[k2]
        structure = structure[0:left_position] + '.' + structure[left_position+1:]      # Correct the right paired base to an unpaired base in the complete sequence
    
    # Correct the structure of targeting region to an unwinded state ready for hybridization to crRNA
    if i == 0:
        structure = '....................' + structure[20:] 
    else:
        structure = structure[0: i*20] + '....................' + structure[(i+1)*20:]
        
    return structure

# We adopted a weighted model to compute the free energy change of the targeting region of a long strand
# To be specific, the free energy change of targeting region in each suboptimal structure is individually considered and the ensemble energy change is the weighted sum of all energy changes in suboptimal structures
def main(sequence, unwind_domain):
    my_model = Model(material='DNA', ensemble='stacking', celsius=37, sodium=0.1, magnesium=0.01)   # Define NUPACK model, ion concentration in consistent with NEBuffer r3.1
    partition_function = pfunc(strands=sequence, model=my_model)
    mfe_structures = mfe(strands=sequence, model=my_model)    # Compute MFE structure of long strand

    subopt_structures = subopt(strands=sequence, energy_gap=1.77, model=my_model)   # Compute all suboptimal structures whose energies are close to the MFE (not exceeding 1.77 kcal/mol and in this case 99% of the suboptimal structures will be considered)
    prob_sum = 0   # Initialize the sum of frequency of a thermodynamic ensemble
    sum_weighted_DeltaG_unwind = 0   # Initialize the weighted sum of free energy change of targeting region 
    for i in range (0, len(subopt_structures)):
        # Compute the suboptimal structure 
        subopt_struct_candidate = str(subopt_structures[i].structure)   
        # Compute the energy under this suboptimal structure 
        subopt_energy_candidate = subopt_structures[i].energy  
        # Compute the frequency of this suboptimal structure 
        probability = structure_probability(strands=sequence, structure=subopt_struct_candidate, model=my_model)
        
        prob_sum += probability  # Frequency sum up
        # Compute suboptimal structure with the targeting region unwinded 
        subopt_struct_unwind = str(DNA_unwind_state(subopt_struct_candidate, unwind_domain))
        # Compute the energy of this suboptimal structure with the targeting region unwinded 
        subopt_energy_unwind = structure_energy(strands=sequence, structure=subopt_struct_unwind, model=my_model)
        
        # The free energy change is the difference of energy between the original state and the unwinded state
        deltaG_unwind = subopt_energy_candidate - subopt_energy_unwind
        # This energy change should be weighted with consideration of the frequency of this suboptimal structure 
        weighted_deltaG_unwind = deltaG_unwind * probability
        # Weighted free energy change sum up
        sum_weighted_DeltaG_unwind += weighted_deltaG_unwind
    
    print('This is domain: ' + str(unwind_domain))    # Refer to the index of target in a long strand
    print("Targeting region sequence: " + str(sequence[unwind_domain*20:(unwind_domain+1)*20]))
    print('Probability sum of all suboptimal strctures searched: ' + str(prob_sum)) 
    print('Weighted sum of energy needed to unwind this domain: ' + str(sum_weighted_DeltaG_unwind) + " kcal/mol")

# All the sequences of long ssDNA used
long_DNA_7_pair = 'GGAATTTCAAATGAAATTCAAGAGTAGTCATCACTACTCATGCAGGACAAGTGCCCTACGCGTCGGGCAGCTGCCCGACCATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG'

# Return the results
print('Long ssDNA target sequence: ' + str(long_DNA_7_pair))
for i in range (0, 8):
    main(long_DNA_7_pair, i)
        

Long ssDNA target sequence: GGAATTTCAAATGAAATTCAAGAGTAGTCATCACTACTCATGCAGGACAAGTGCCCTACGCGTCGGGCAGCTGCCCGACCATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG
This is domain: 0
Targeting region sequence: GGAATTTCAAATGAAATTCA
Probability sum of all suboptimal strctures searched: 0.9974169779461222
Weighted sum of energy needed to unwind this domain: -5.025769080401654 kcal/mol
This is domain: 1
Targeting region sequence: AGAGTAGTCATCACTACTCA
Probability sum of all suboptimal strctures searched: 0.9974169779461222
Weighted sum of energy needed to unwind this domain: -5.075201570467633 kcal/mol
This is domain: 2
Targeting region sequence: TGCAGGACAAGTGCCCTACG
Probability sum of all suboptimal strctures searched: 0.9974169779461222
Weighted sum of energy needed to unwind this domain: -0.7572791940790556 kcal/mol
This is domain: 3
Targeting region sequence: CGTCGGGCAGCTGCCCGACC
Probability sum of all suboptimal strctures searched: 0.9974169779461222
Weighted su

In [94]:
# Compute free energy change of 6_pair long ssDNA activator (160 nt)
# It should be noted that in a computed structure of a targeting region, the left hybridized base may not always correspond to a right hybridized base as it may hybridize with a base outside of the targeting region
# In a computed structure, '(' refers to a left paired base and ")" refers to a right paired base

from nupack import *   # Import NUPACK package 

# Count the number of left base pairs and right base pairs in a long ssDNA structure
def paren_num(structure):
    left_paren_num = 0
    right_paren_num = 0
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_num += 1
        if structure[i] == ')':
            right_paren_num += 1
            
    return left_paren_num, right_paren_num

# Record the position of left paired bases and right paired bases in a long ssDNA structure
def paren_position(structure):
    left_paren = []
    right_paren = []
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            
    return left_paren, right_paren
        
# Record positions of base pairs with the order of base positions in "left_paren" corresponding to the base positions in "right_paren"
def find_pair(structure):
    left_paren = []
    right_paren = []
    left_paren_pre = []

    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_pre.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            left_paren.append(left_paren_pre[-1])
            left_paren_pre = left_paren_pre[:-1]

    return left_paren, right_paren

# 
def DNA_unwind_state(structure, i):
    left_paren, right_paren = find_pair(structure)
    ori_structure_part = structure[i*20:(i+1)*20]    # Compute the original structure of targeting region
    left_paren_num_part = paren_num(ori_structure_part)[0]   # Get the number of left base pairs
    right_paren_num_part = paren_num(ori_structure_part)[1]  # Get the number of right base pairs
    left_paren_part = paren_position(ori_structure_part)[0]  # Get the relative positions of left paired bases in 20-nt target
    right_paren_part = paren_position(ori_structure_part)[1] # Get the relative positions of right paired bases in 20-nt target

    left_paren_part = [x+i*20 for x in left_paren_part]      # Get the absolute positions of left paired bases 
    right_paren_part = [x+i*20 for x in right_paren_part]    # Get the absolute positions of right paired bases 
    left_paren_part = set(left_paren_part)                   # Arrange the order and eliminate repeats 
    right_paren_part = set(right_paren_part)
    index_left = [m for m, n in enumerate(left_paren) if n in left_paren_part]     # Get the order of left paired bases in the complete long ssDNA sequence
    index_right = [m for m, n in enumerate(right_paren) if n in right_paren_part]  # Get the order of right paired bases in the complete long ssDNA sequence
    for k1 in index_left:
        right_position = right_paren[k1]
        structure = structure[0:right_position] + '.' + structure[right_position+1:]    # Correct the left paired base to an unpaired base in the complete sequence
    for k2 in index_right:
        left_position = left_paren[k2]
        structure = structure[0:left_position] + '.' + structure[left_position+1:]      # Correct the right paired base to an unpaired base in the complete sequence
    
    # Correct the structure of targeting region to an unwinded state ready for hybridization to crRNA
    if i == 0:
        structure = '....................' + structure[20:] 
    else:
        structure = structure[0: i*20] + '....................' + structure[(i+1)*20:]
        
    return structure

# We adopted a weighted model to compute the free energy change of the targeting region of a long strand
# To be specific, the free energy change of targeting region in each suboptimal structure is individually considered and the ensemble energy change is the weighted sum of all energy changes in suboptimal structures
def main(sequence, unwind_domain):
    my_model = Model(material='DNA', ensemble='stacking', celsius=37, sodium=0.1, magnesium=0.01)   # Define NUPACK model, ion concentration in consistent with NEBuffer r3.1
    partition_function = pfunc(strands=sequence, model=my_model)
    mfe_structures = mfe(strands=sequence, model=my_model)    # Compute MFE structure of long strand

    subopt_structures = subopt(strands=sequence, energy_gap=1.605, model=my_model)   # Compute all suboptimal structures whose energies are close to the MFE (not exceeding 1.605 kcal/mol and in this case 99% of the suboptimal structures will be considered)
    prob_sum = 0   # Initialize the sum of frequency of a thermodynamic ensemble
    sum_weighted_DeltaG_unwind = 0   # Initialize the weighted sum of free energy change of targeting region 
    for i in range (0, len(subopt_structures)):
        # Compute the suboptimal structure 
        subopt_struct_candidate = str(subopt_structures[i].structure)   
        # Compute the energy under this suboptimal structure 
        subopt_energy_candidate = subopt_structures[i].energy  
        # Compute the frequency of this suboptimal structure 
        probability = structure_probability(strands=sequence, structure=subopt_struct_candidate, model=my_model)
        
        prob_sum += probability  # Frequency sum up
        # Compute suboptimal structure with the targeting region unwinded 
        subopt_struct_unwind = str(DNA_unwind_state(subopt_struct_candidate, unwind_domain))
        # Compute the energy of this suboptimal structure with the targeting region unwinded 
        subopt_energy_unwind = structure_energy(strands=sequence, structure=subopt_struct_unwind, model=my_model)
        
        # The free energy change is the difference of energy between the original state and the unwinded state
        deltaG_unwind = subopt_energy_candidate - subopt_energy_unwind
        # This energy change should be weighted with consideration of the frequency of this suboptimal structure 
        weighted_deltaG_unwind = deltaG_unwind * probability
        # Weighted free energy change sum up
        sum_weighted_DeltaG_unwind += weighted_deltaG_unwind
    
    print('This is domain: ' + str(unwind_domain))    # Refer to the index of target in a long strand
    print("Targeting region sequence: " + str(sequence[unwind_domain*20:(unwind_domain+1)*20]))
    print('Probability sum of all suboptimal strctures searched: ' + str(prob_sum)) 
    print('Weighted sum of energy needed to unwind this domain: ' + str(sum_weighted_DeltaG_unwind) + " kcal/mol")

# All the sequences of long ssDNA used
long_DNA_6_pair = 'TTGATTTCATTTGAAATCTGGAGTCACTTATCAGTAACGATGGCCAAGGCAACTTGACGGGAGGCTCGTGGCCGAGCCGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG'

# Return the results
print('Long ssDNA target sequence: ' + str(long_DNA_6_pair))
for i in range (0, 8):
    main(long_DNA_6_pair, i)
        

Long ssDNA target sequence: TTGATTTCATTTGAAATCTGGAGTCACTTATCAGTAACGATGGCCAAGGCAACTTGACGGGAGGCTCGTGGCCGAGCCGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG
This is domain: 0
Targeting region sequence: TTGATTTCATTTGAAATCTG
Probability sum of all suboptimal strctures searched: 0.9908999430704355
Weighted sum of energy needed to unwind this domain: -3.880762475124494 kcal/mol
This is domain: 1
Targeting region sequence: GAGTCACTTATCAGTAACGA
Probability sum of all suboptimal strctures searched: 0.9908999430704355
Weighted sum of energy needed to unwind this domain: -0.36156886322778986 kcal/mol
This is domain: 2
Targeting region sequence: TGGCCAAGGCAACTTGACGG
Probability sum of all suboptimal strctures searched: 0.9908999430704355
Weighted sum of energy needed to unwind this domain: -4.479623796278323 kcal/mol
This is domain: 3
Targeting region sequence: GAGGCTCGTGGCCGAGCCGG
Probability sum of all suboptimal strctures searched: 0.9908999430704355
Weighted s

In [95]:
# Compute free energy change of 5_pair long ssDNA activator (160 nt)
# It should be noted that in a computed structure of a targeting region, the left hybridized base may not always correspond to a right hybridized base as it may hybridize with a base outside of the targeting region
# In a computed structure, '(' refers to a left paired base and ")" refers to a right paired base

from nupack import *   # Import NUPACK package 

# Count the number of left base pairs and right base pairs in a long ssDNA structure
def paren_num(structure):
    left_paren_num = 0
    right_paren_num = 0
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_num += 1
        if structure[i] == ')':
            right_paren_num += 1
            
    return left_paren_num, right_paren_num

# Record the position of left paired bases and right paired bases in a long ssDNA structure
def paren_position(structure):
    left_paren = []
    right_paren = []
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            
    return left_paren, right_paren
        
# Record positions of base pairs with the order of base positions in "left_paren" corresponding to the base positions in "right_paren"
def find_pair(structure):
    left_paren = []
    right_paren = []
    left_paren_pre = []

    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_pre.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            left_paren.append(left_paren_pre[-1])
            left_paren_pre = left_paren_pre[:-1]

    return left_paren, right_paren

# 
def DNA_unwind_state(structure, i):
    left_paren, right_paren = find_pair(structure)
    ori_structure_part = structure[i*20:(i+1)*20]    # Compute the original structure of targeting region
    left_paren_num_part = paren_num(ori_structure_part)[0]   # Get the number of left base pairs
    right_paren_num_part = paren_num(ori_structure_part)[1]  # Get the number of right base pairs
    left_paren_part = paren_position(ori_structure_part)[0]  # Get the relative positions of left paired bases in 20-nt target
    right_paren_part = paren_position(ori_structure_part)[1] # Get the relative positions of right paired bases in 20-nt target

    left_paren_part = [x+i*20 for x in left_paren_part]      # Get the absolute positions of left paired bases 
    right_paren_part = [x+i*20 for x in right_paren_part]    # Get the absolute positions of right paired bases 
    left_paren_part = set(left_paren_part)                   # Arrange the order and eliminate repeats 
    right_paren_part = set(right_paren_part)
    index_left = [m for m, n in enumerate(left_paren) if n in left_paren_part]     # Get the order of left paired bases in the complete long ssDNA sequence
    index_right = [m for m, n in enumerate(right_paren) if n in right_paren_part]  # Get the order of right paired bases in the complete long ssDNA sequence
    for k1 in index_left:
        right_position = right_paren[k1]
        structure = structure[0:right_position] + '.' + structure[right_position+1:]    # Correct the left paired base to an unpaired base in the complete sequence
    for k2 in index_right:
        left_position = left_paren[k2]
        structure = structure[0:left_position] + '.' + structure[left_position+1:]      # Correct the right paired base to an unpaired base in the complete sequence
    
    # Correct the structure of targeting region to an unwinded state ready for hybridization to crRNA
    if i == 0:
        structure = '....................' + structure[20:] 
    else:
        structure = structure[0: i*20] + '....................' + structure[(i+1)*20:]
        
    return structure

# We adopted a weighted model to compute the free energy change of the targeting region of a long strand
# To be specific, the free energy change of targeting region in each suboptimal structure is individually considered and the ensemble energy change is the weighted sum of all energy changes in suboptimal structures
def main(sequence, unwind_domain):
    my_model = Model(material='DNA', ensemble='stacking', celsius=37, sodium=0.1, magnesium=0.01)   # Define NUPACK model, ion concentration in consistent with NEBuffer r3.1
    partition_function = pfunc(strands=sequence, model=my_model)
    mfe_structures = mfe(strands=sequence, model=my_model)    # Compute MFE structure of long strand

    subopt_structures = subopt(strands=sequence, energy_gap=2.52, model=my_model)   # Compute all suboptimal structures whose energies are close to the MFE (not exceeding 2.52 kcal/mol and in this case 99% of the suboptimal structures will be considered)
    prob_sum = 0   # Initialize the sum of frequency of a thermodynamic ensemble
    sum_weighted_DeltaG_unwind = 0   # Initialize the weighted sum of free energy change of targeting region 
    for i in range (0, len(subopt_structures)):
        # Compute the suboptimal structure 
        subopt_struct_candidate = str(subopt_structures[i].structure)   
        # Compute the energy under this suboptimal structure 
        subopt_energy_candidate = subopt_structures[i].energy  
        # Compute the frequency of this suboptimal structure 
        probability = structure_probability(strands=sequence, structure=subopt_struct_candidate, model=my_model)
        
        prob_sum += probability  # Frequency sum up
        # Compute suboptimal structure with the targeting region unwinded 
        subopt_struct_unwind = str(DNA_unwind_state(subopt_struct_candidate, unwind_domain))
        # Compute the energy of this suboptimal structure with the targeting region unwinded 
        subopt_energy_unwind = structure_energy(strands=sequence, structure=subopt_struct_unwind, model=my_model)
        
        # The free energy change is the difference of energy between the original state and the unwinded state
        deltaG_unwind = subopt_energy_candidate - subopt_energy_unwind
        # This energy change should be weighted with consideration of the frequency of this suboptimal structure 
        weighted_deltaG_unwind = deltaG_unwind * probability
        # Weighted free energy change sum up
        sum_weighted_DeltaG_unwind += weighted_deltaG_unwind
    
    print('This is domain: ' + str(unwind_domain))    # Refer to the index of target in a long strand
    print("Targeting region sequence: " + str(sequence[unwind_domain*20:(unwind_domain+1)*20]))
    print('Probability sum of all suboptimal strctures searched: ' + str(prob_sum)) 
    print('Weighted sum of energy needed to unwind this domain: ' + str(sum_weighted_DeltaG_unwind) + " kcal/mol")

# All the sequences of long ssDNA used
long_DNA_5_pair = 'AAACATTCAATAGAATGAGAAGGGCTAGAATACTAACTTGGTTGCGACCTCTGTCGCTTGCGAGCGCCTGCAGGCACGGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG'

# Return the results
print('Long ssDNA target sequence: ' + str(long_DNA_5_pair))
for i in range (0, 8):
    main(long_DNA_5_pair, i)
        

Long ssDNA target sequence: AAACATTCAATAGAATGAGAAGGGCTAGAATACTAACTTGGTTGCGACCTCTGTCGCTTGCGAGCGCCTGCAGGCACGGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG
This is domain: 0
Targeting region sequence: AAACATTCAATAGAATGAGA
Probability sum of all suboptimal strctures searched: 0.9962555473689895
Weighted sum of energy needed to unwind this domain: -3.9906454233310806 kcal/mol
This is domain: 1
Targeting region sequence: AGGGCTAGAATACTAACTTG
Probability sum of all suboptimal strctures searched: 0.9962555473689895
Weighted sum of energy needed to unwind this domain: 0.053418219798324854 kcal/mol
This is domain: 2
Targeting region sequence: GTTGCGACCTCTGTCGCTTG
Probability sum of all suboptimal strctures searched: 0.9962555473689895
Weighted sum of energy needed to unwind this domain: -5.605370612693789 kcal/mol
This is domain: 3
Targeting region sequence: CGAGCGCCTGCAGGCACGGG
Probability sum of all suboptimal strctures searched: 0.9962555473689895
Weighted 

In [96]:
# Compute free energy change of 4_pair long ssDNA activator (160 nt)
# It should be noted that in a computed structure of a targeting region, the left hybridized base may not always correspond to a right hybridized base as it may hybridize with a base outside of the targeting region
# In a computed structure, '(' refers to a left paired base and ")" refers to a right paired base

from nupack import *   # Import NUPACK package 

# Count the number of left base pairs and right base pairs in a long ssDNA structure
def paren_num(structure):
    left_paren_num = 0
    right_paren_num = 0
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_num += 1
        if structure[i] == ')':
            right_paren_num += 1
            
    return left_paren_num, right_paren_num

# Record the position of left paired bases and right paired bases in a long ssDNA structure
def paren_position(structure):
    left_paren = []
    right_paren = []
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            
    return left_paren, right_paren
        
# Record positions of base pairs with the order of base positions in "left_paren" corresponding to the base positions in "right_paren"
def find_pair(structure):
    left_paren = []
    right_paren = []
    left_paren_pre = []

    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_pre.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            left_paren.append(left_paren_pre[-1])
            left_paren_pre = left_paren_pre[:-1]

    return left_paren, right_paren

# 
def DNA_unwind_state(structure, i):
    left_paren, right_paren = find_pair(structure)
    ori_structure_part = structure[i*20:(i+1)*20]    # Compute the original structure of targeting region
    left_paren_num_part = paren_num(ori_structure_part)[0]   # Get the number of left base pairs
    right_paren_num_part = paren_num(ori_structure_part)[1]  # Get the number of right base pairs
    left_paren_part = paren_position(ori_structure_part)[0]  # Get the relative positions of left paired bases in 20-nt target
    right_paren_part = paren_position(ori_structure_part)[1] # Get the relative positions of right paired bases in 20-nt target

    left_paren_part = [x+i*20 for x in left_paren_part]      # Get the absolute positions of left paired bases 
    right_paren_part = [x+i*20 for x in right_paren_part]    # Get the absolute positions of right paired bases 
    left_paren_part = set(left_paren_part)                   # Arrange the order and eliminate repeats 
    right_paren_part = set(right_paren_part)
    index_left = [m for m, n in enumerate(left_paren) if n in left_paren_part]     # Get the order of left paired bases in the complete long ssDNA sequence
    index_right = [m for m, n in enumerate(right_paren) if n in right_paren_part]  # Get the order of right paired bases in the complete long ssDNA sequence
    for k1 in index_left:
        right_position = right_paren[k1]
        structure = structure[0:right_position] + '.' + structure[right_position+1:]    # Correct the left paired base to an unpaired base in the complete sequence
    for k2 in index_right:
        left_position = left_paren[k2]
        structure = structure[0:left_position] + '.' + structure[left_position+1:]      # Correct the right paired base to an unpaired base in the complete sequence
    
    # Correct the structure of targeting region to an unwinded state ready for hybridization to crRNA
    if i == 0:
        structure = '....................' + structure[20:] 
    else:
        structure = structure[0: i*20] + '....................' + structure[(i+1)*20:]
        
    return structure

# We adopted a weighted model to compute the free energy change of the targeting region of a long strand
# To be specific, the free energy change of targeting region in each suboptimal structure is individually considered and the ensemble energy change is the weighted sum of all energy changes in suboptimal structures
def main(sequence, unwind_domain):
    my_model = Model(material='DNA', ensemble='stacking', celsius=37, sodium=0.1, magnesium=0.01)   # Define NUPACK model, ion concentration in consistent with NEBuffer r3.1
    partition_function = pfunc(strands=sequence, model=my_model)
    mfe_structures = mfe(strands=sequence, model=my_model)    # Compute MFE structure of long strand

    subopt_structures = subopt(strands=sequence, energy_gap=2.14, model=my_model)   # Compute all suboptimal structures whose energies are close to the MFE (not exceeding 2.14 kcal/mol and in this case 99% of the suboptimal structures will be considered)
    prob_sum = 0   # Initialize the sum of frequency of a thermodynamic ensemble
    sum_weighted_DeltaG_unwind = 0   # Initialize the weighted sum of free energy change of targeting region 
    for i in range (0, len(subopt_structures)):
        # Compute the suboptimal structure 
        subopt_struct_candidate = str(subopt_structures[i].structure)   
        # Compute the energy under this suboptimal structure 
        subopt_energy_candidate = subopt_structures[i].energy  
        # Compute the frequency of this suboptimal structure 
        probability = structure_probability(strands=sequence, structure=subopt_struct_candidate, model=my_model)
        
        prob_sum += probability  # Frequency sum up
        # Compute suboptimal structure with the targeting region unwinded 
        subopt_struct_unwind = str(DNA_unwind_state(subopt_struct_candidate, unwind_domain))
        # Compute the energy of this suboptimal structure with the targeting region unwinded 
        subopt_energy_unwind = structure_energy(strands=sequence, structure=subopt_struct_unwind, model=my_model)
        
        # The free energy change is the difference of energy between the original state and the unwinded state
        deltaG_unwind = subopt_energy_candidate - subopt_energy_unwind
        # This energy change should be weighted with consideration of the frequency of this suboptimal structure 
        weighted_deltaG_unwind = deltaG_unwind * probability
        # Weighted free energy change sum up
        sum_weighted_DeltaG_unwind += weighted_deltaG_unwind
    
    print('This is domain: ' + str(unwind_domain))    # Refer to the index of target in a long strand
    print("Targeting region sequence: " + str(sequence[unwind_domain*20:(unwind_domain+1)*20]))
    print('Probability sum of all suboptimal strctures searched: ' + str(prob_sum)) 
    print('Weighted sum of energy needed to unwind this domain: ' + str(sum_weighted_DeltaG_unwind) + " kcal/mol")

# All the sequences of long ssDNA used
long_DNA_4_pair = 'TATTGTACTATAGTACTTTGATTGGCAGAATACTACGTGATTATGCCTCGCCAGGCTGGAGCGTGGACCGTGGCCCTGGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG'

# Return the results
print('Long ssDNA target sequence: ' + str(long_DNA_4_pair))
for i in range (0, 8):
    main(long_DNA_4_pair, i)
        

Long ssDNA target sequence: TATTGTACTATAGTACTTTGATTGGCAGAATACTACGTGATTATGCCTCGCCAGGCTGGAGCGTGGACCGTGGCCCTGGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG
This is domain: 0
Targeting region sequence: TATTGTACTATAGTACTTTG
Probability sum of all suboptimal strctures searched: 0.9977645021938798
Weighted sum of energy needed to unwind this domain: -2.114462924607845 kcal/mol
This is domain: 1
Targeting region sequence: ATTGGCAGAATACTACGTGA
Probability sum of all suboptimal strctures searched: 0.9977645021938798
Weighted sum of energy needed to unwind this domain: -3.9704144077315626 kcal/mol
This is domain: 2
Targeting region sequence: TTATGCCTCGCCAGGCTGGA
Probability sum of all suboptimal strctures searched: 0.9977645021938798
Weighted sum of energy needed to unwind this domain: -11.139656930195386 kcal/mol
This is domain: 3
Targeting region sequence: GCGTGGACCGTGGCCCTGGG
Probability sum of all suboptimal strctures searched: 0.9977645021938798
Weighted s

In [93]:
# Compute free energy change of 3_pair long ssDNA activator (160 nt)
# It should be noted that in a computed structure of a targeting region, the left hybridized base may not always correspond to a right hybridized base as it may hybridize with a base outside of the targeting region
# In a computed structure, '(' refers to a left paired base and ")" refers to a right paired base

from nupack import *   # Import NUPACK package 

# Count the number of left base pairs and right base pairs in a long ssDNA structure
def paren_num(structure):
    left_paren_num = 0
    right_paren_num = 0
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_num += 1
        if structure[i] == ')':
            right_paren_num += 1
            
    return left_paren_num, right_paren_num

# Record the position of left paired bases and right paired bases in a long ssDNA structure
def paren_position(structure):
    left_paren = []
    right_paren = []
    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            
    return left_paren, right_paren
        
# Record positions of base pairs with the order of base positions in "left_paren" corresponding to the base positions in "right_paren"
def find_pair(structure):
    left_paren = []
    right_paren = []
    left_paren_pre = []

    for i in range (0, len(structure)):
        if structure[i] == '(':
            left_paren_pre.append(i)
        if structure[i] == ')':
            right_paren.append(i)
            left_paren.append(left_paren_pre[-1])
            left_paren_pre = left_paren_pre[:-1]

    return left_paren, right_paren

# 
def DNA_unwind_state(structure, i):
    left_paren, right_paren = find_pair(structure)
    ori_structure_part = structure[i*20:(i+1)*20]    # Compute the original structure of targeting region
    left_paren_num_part = paren_num(ori_structure_part)[0]   # Get the number of left base pairs
    right_paren_num_part = paren_num(ori_structure_part)[1]  # Get the number of right base pairs
    left_paren_part = paren_position(ori_structure_part)[0]  # Get the relative positions of left paired bases in 20-nt target
    right_paren_part = paren_position(ori_structure_part)[1] # Get the relative positions of right paired bases in 20-nt target

    left_paren_part = [x+i*20 for x in left_paren_part]      # Get the absolute positions of left paired bases 
    right_paren_part = [x+i*20 for x in right_paren_part]    # Get the absolute positions of right paired bases 
    left_paren_part = set(left_paren_part)                   # Arrange the order and eliminate repeats 
    right_paren_part = set(right_paren_part)
    index_left = [m for m, n in enumerate(left_paren) if n in left_paren_part]     # Get the order of left paired bases in the complete long ssDNA sequence
    index_right = [m for m, n in enumerate(right_paren) if n in right_paren_part]  # Get the order of right paired bases in the complete long ssDNA sequence
    for k1 in index_left:
        right_position = right_paren[k1]
        structure = structure[0:right_position] + '.' + structure[right_position+1:]    # Correct the left paired base to an unpaired base in the complete sequence
    for k2 in index_right:
        left_position = left_paren[k2]
        structure = structure[0:left_position] + '.' + structure[left_position+1:]      # Correct the right paired base to an unpaired base in the complete sequence
    
    # Correct the structure of targeting region to an unwinded state ready for hybridization to crRNA
    if i == 0:
        structure = '....................' + structure[20:] 
    else:
        structure = structure[0: i*20] + '....................' + structure[(i+1)*20:]
        
    return structure

# We adopted a weighted model to compute the free energy change of the targeting region of a long strand
# To be specific, the free energy change of targeting region in each suboptimal structure is individually considered and the ensemble energy change is the weighted sum of all energy changes in suboptimal structures
def main(sequence, unwind_domain):
    my_model = Model(material='DNA', ensemble='stacking', celsius=37, sodium=0.1, magnesium=0.01)   # Define NUPACK model, ion concentration in consistent with NEBuffer r3.1
    partition_function = pfunc(strands=sequence, model=my_model)
    mfe_structures = mfe(strands=sequence, model=my_model)    # Compute MFE structure of long strand

    subopt_structures = subopt(strands=sequence, energy_gap=2.02, model=my_model)   # Compute all suboptimal structures whose energies are close to the MFE (not exceeding 2.02 kcal/mol and in this case 61% of the suboptimal structures will be considered)
    prob_sum = 0   # Initialize the sum of frequency of a thermodynamic ensemble
    sum_weighted_DeltaG_unwind = 0   # Initialize the weighted sum of free energy change of targeting region 
    for i in range (0, len(subopt_structures)):
        # Compute the suboptimal structure 
        subopt_struct_candidate = str(subopt_structures[i].structure)   
        # Compute the energy under this suboptimal structure 
        subopt_energy_candidate = subopt_structures[i].energy  
        # Compute the frequency of this suboptimal structure 
        probability = structure_probability(strands=sequence, structure=subopt_struct_candidate, model=my_model)
        
        prob_sum += probability  # Frequency sum up
        # Compute suboptimal structure with the targeting region unwinded 
        subopt_struct_unwind = str(DNA_unwind_state(subopt_struct_candidate, unwind_domain))
        # Compute the energy of this suboptimal structure with the targeting region unwinded 
        subopt_energy_unwind = structure_energy(strands=sequence, structure=subopt_struct_unwind, model=my_model)
        
        # The free energy change is the difference of energy between the original state and the unwinded state
        deltaG_unwind = subopt_energy_candidate - subopt_energy_unwind
        # This energy change should be weighted with consideration of the frequency of this suboptimal structure 
        weighted_deltaG_unwind = deltaG_unwind * probability
        # Weighted free energy change sum up
        sum_weighted_DeltaG_unwind += weighted_deltaG_unwind
    
    print('This is domain: ' + str(unwind_domain))    # Refer to the index of target in a long strand
    print("Targeting region sequence: " + str(sequence[unwind_domain*20:(unwind_domain+1)*20]))
    print('Probability sum of all suboptimal strctures searched: ' + str(prob_sum)) 
    
    # Since the weighted energy just occupies 61% of the ensemble, we divide it by the prob_sum to estimate the actual ensemble free energy change
    sum_weighted_DeltaG_unwind = sum_weighted_DeltaG_unwind/prob_sum
    print('Weighted sum of energy needed to unwind this domain: ' + str(sum_weighted_DeltaG_unwind) + " kcal/mol")

# All the sequences of long ssDNA used
long_DNA_3_pair = 'AATTTGACATTAGTCTTTGAATATTGTCATAAGACGGTCGATGATGGCGGTTGCCGAGGACGCGTGGCAGGTGCCGTGGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG'

# Return the results
print('Long ssDNA target sequence: ' + str(long_DNA_3_pair))
for i in range (0, 8):
    main(long_DNA_3_pair, i)
        

Long ssDNA target sequence: AATTTGACATTAGTCTTTGAATATTGTCATAAGACGGTCGATGATGGCGGTTGCCGAGGACGCGTGGCAGGTGCCGTGGGATATTGTTATTGGATTTGGAAGTAAATGGAAATGGTGAGGCTGGGATGGGATTAGCTGGGCGGGAGCAGGCTGGGTCGGG
This is domain: 0
Targeting region sequence: AATTTGACATTAGTCTTTGA
Probability sum of all suboptimal strctures searched: 0.6148937279584469
Weighted sum of energy needed to unwind this domain: -2.0119131042632876 kcal/mol
This is domain: 1
Targeting region sequence: ATATTGTCATAAGACGGTCG
Probability sum of all suboptimal strctures searched: 0.6148937279584469
Weighted sum of energy needed to unwind this domain: -2.9671472144616216 kcal/mol
This is domain: 2
Targeting region sequence: ATGATGGCGGTTGCCGAGGA
Probability sum of all suboptimal strctures searched: 0.6148937279584469
Weighted sum of energy needed to unwind this domain: -2.489372440895625 kcal/mol
This is domain: 3
Targeting region sequence: CGCGTGGCAGGTGCCGTGGG
Probability sum of all suboptimal strctures searched: 0.6148937279584469
Weighted s